In [ ]:
%matplotlib inline
# %matplotlib

In [ ]:
# Loading libaries

import datetime
import os
import pickle
import time

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors 
from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection

from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('png')

plt.rcParams['savefig.dpi'] = 75
plt.rcParams['figure.autolayout'] = False
plt.rcParams['figure.figsize'] = (21, 30)  # A4
plt.rcParams['lines.linewidth'] = 1.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['text.usetex'] = False
plt.rcParams['font.size'] = 12
plt.rcParams['font.family'] = "sans-serif"
matplotlib.rcParams['xtick.major.pad']=12
matplotlib.rcParams['ytick.major.pad']=12
#matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

import numpy as np
import scipy as scipy
import scipy.stats as stats
import scipy.linalg as linalg
import xarray as xr
from pprint import pprint
import sys
import os
import copy
import string
import glob
#import xarray as xr 

import warnings
#warnings.filterwarnings('ignore')

In [ ]:
base_path = os.path.abspath('L63_FTLE_Caputo.ipynb')
print('base path', base_path)

In [ ]:
now = datetime.datetime.now()
print('date : {0} / {1} / {2}'.format(now.day, now.month, now.year))
cwd = os.getcwd()
print('working directory : {0}'.format(cwd))
print('data directory : {0}'.format(cwd))

In [ ]:
from DA_module_caputo import *
from ftles import *
from CLVs import *

# Control Run

In [ ]:
# Model properties
dt                = 0.01

parameters_spinup = 50     # skip this first number of analyses in determining statistics
forecast_window   = 25      # window between each DA cycle

dim  = 5

# control run
num_steps = 100000
np.random.seed(seed=1)
IC     = np.ones(dim)
#IC_run = Lorenz63(num_time_steps=num_steps, dt=dt, IC=IC)
IC_run = Caputo(num_time_steps=num_steps, dt=dt, IC=[1.0, 0.0, 0.0, 0.0, 0.0])
#IC_run = Threescroll(num_time_steps=num_steps, dt=dt, IC=[1.618,-1.618,1.618])
IC_run.run()

In [ ]:
fig, a1 = plt.subplots(1,1,figsize=[20,10])
a1.plot(IC_run.time[:20000],IC_run.x[:,:20000].T)
a1.grid()
a1.set_xlabel('time')
plt.show()

In [ ]:
fig = plt.figure(figsize=[10,10])
a1 = fig.add_subplot(1,1,1,projection='3d')
a1.plot(IC_run.x[0,30000:],IC_run.x[2,30000:],IC_run.x[3,30000:],'.',ms=0.5)
a1.view_init(elev=30, azim=120)  # Set elevation and azimuth angles
a1.grid()
a1.set_xlabel('x')
a1.set_ylabel('y')
a1.set_zlabel('z')
plt.show()

In [ ]:

fig, a1 = plt.subplots(1,1,figsize=[20,10])
a1.plot(IC_run.x[1,:20000],IC_run.x[3,:20000])
a1.grid()
a1.set_xlabel('time')
plt.show()

In [ ]:
fig = plt.figure(figsize=[24,30],layout='constrained',dpi=300)
dim2 = 5
for i in np.arange(0,dim2):
    for j in np.arange(0,i):
        a1 = fig.add_subplot(dim2,dim2,i*dim2+j+1)
        a1.plot(IC_run.x[i,:20000],IC_run.x[j,:20000])
        a1.grid()
        a1.set_title('{}-{} axes'.format(i+1,j+1))
plt.savefig('Results/Caputo/2D axes view.png',format="png",bbox_inches='tight')
plt.show()  

# Dynamical Properties

In [ ]:
start_ind = 4000
start_time = dt*start_ind

start = time.time()
cocycle = IC_run.calculate_matrix_cocycle(IC_run.x,tlims=[start_time,dt*num_steps])
end = time.time()

comp_time = (end-start)

print('cocycle calculated in ' + str(np.round(comp_time,4)) + ' sec')

## FTLEs

In [ ]:
CLVS_DIR = os.path.join(cwd, 'CLVs')
print('dim', dim, 'shape=', cocycle.shape[2])
if not os.path.exists(CLVS_DIR):
    os.makedirs(CLVS_DIR)

Rdiag_filename = '.'.join(['CaputoRdiag','nc'])
Rdiag_file = os.path.join(CLVS_DIR, Rdiag_filename)

#if os.path.exists(Rdiag_file):
#    Rdiag = xr.open_dataset(Rdiag_file).Rdiag
#else:
start = time.time()
Rdiag = calculate_FTLEs(dim, cocycle, np.arange(0,cocycle.shape[2]))[1]
end = time.time()

comp_time = (end-start)/60

print('R matrices calculated in ' + str(np.round(comp_time,4)) + ' min')
    
    ## convert to xarray
Rdiag = xr.DataArray(Rdiag)
    
    # save to netCDF
Rdiag.to_dataset(name='CaputoRdiag').to_netcdf(Rdiag_file)

In [ ]:
window = 5

FTLEs = np.zeros((dim,num_steps-start_ind-window))

start = time.time()
for ti in np.arange(0,num_steps-window-start_ind):
    FTLEs[:,ti] = np.sum(np.log(Rdiag[:,ti:ti+window]),axis=1)/(window*dt)
end = time.time()
comp_time = (end-start)

print('calculated in ' + str(np.round(comp_time,4)) + ' sec')

In [ ]:
mycmap = matplotlib.colors.ListedColormap([
 [0., 0., 1., 1.      ],
 [0.25, 0.25, 1., 1.      ],
 [0.5, 0.5, 1., 1.      ], 
 [0.75, 0.75, 1., 1.      ],
 [1., 0.75, 0.75, 1.      ],
 [1., 0.5, 0.5, 1.      ],
 [1., 0.25, 0.25, 1.      ],
 [1., 0., 0., 1.      ]])

fig = plt.figure(figsize=[24,10],layout='constrained',dpi=300)
a1 = fig.add_subplot(2,3,1,projection='3d')
scat = a1.scatter(IC_run.x[0,start_ind:-window],IC_run.x[1,start_ind:-window],IC_run.x[2,start_ind:-window],
           c=FTLEs[0,:],s=2,cmap='RdBu_r',vmax=5,vmin=-5)
a1.view_init(elev=30, azim=0)  # Set elevation and azimuth angles
a1.grid()
#a1.set_xlabel('x')
#a1.set_ylabel('y')
#a1.set_zlabel('z')
cbar = plt.colorbar(scat,shrink=0.7)
#cbar.set_label('FTLE 1')
a1.set_title('FTLE 1')

a2 = fig.add_subplot(2,3,2,projection='3d')
scat = a2.scatter(IC_run.x[0,start_ind:-window],IC_run.x[1,start_ind:-window],IC_run.x[2,start_ind:-window],
           c=FTLEs[1,:],s=2,cmap='RdBu_r',vmax=2,vmin=-2)
a2.view_init(elev=30, azim=0)  # Set elevation and azimuth angles
a2.grid()
#a2.set_xlabel('x')
#a2.set_ylabel('y')
#a2.set_zlabel('z')
cbar = plt.colorbar(scat,shrink=0.7)
#cbar.set_label('FTLE 2')
a2.set_title('FTLE 2')

a3 = fig.add_subplot(2,3,3,projection='3d')
scat = a3.scatter(IC_run.x[0,start_ind:-window],IC_run.x[1,start_ind:-window],IC_run.x[2,start_ind:-window],
           c=FTLEs[2,:],s=2,cmap='RdBu_r',vmax=1,vmin=-1)
a3.view_init(elev=30, azim=0)  # Set elevation and azimuth angles
a3.grid()
#a3.set_xlabel('x')
#a3.set_ylabel('y')
#a3.set_zlabel('z')
a3.set_title('FTLE 3')
cbar = plt.colorbar(scat,shrink=0.7)
#cbar.set_label('FTLE 3')

a4 = fig.add_subplot(2,3,4,projection='3d')
scat = a4.scatter(IC_run.x[0,start_ind:-window],IC_run.x[1,start_ind:-window],IC_run.x[2,start_ind:-window],
           c=FTLEs[3,:],s=2,cmap='RdBu_r',vmax=1,vmin=-1)
a4.view_init(elev=30, azim=0)  # Set elevation and azimuth angles
a4.grid()
#a4.set_xlabel('x')
#a4.set_ylabel('y')
#a4.set_zlabel('z')
cbar = plt.colorbar(scat,shrink=0.7)
#cbar.set_label('FTLE 4')
a4.set_title('FTLE 4')

a5 = fig.add_subplot(2,3,5,projection='3d')
scat = a5.scatter(IC_run.x[0,start_ind:-window],IC_run.x[1,start_ind:-window],IC_run.x[2,start_ind:-window],
           c=FTLEs[4,:],s=2,cmap='RdBu_r',vmax=1.5,vmin=-1.5)
a5.view_init(elev=30, azim=0)  # Set elevation and azimuth angles
a5.grid()
#a5.set_xlabel('x')
#a5.set_ylabel('y')
#a5.set_zlabel('z')
cbar = plt.colorbar(scat,shrink=0.7)
#cbar.set_label('FTLE 5')
a5.set_title('FTLE 5')

plt.savefig('Results/Caputo/FTBLEs_wind{}.png'.format(window),format="png",bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=[24,12])
a1 = fig.add_subplot(5,1,1)
scat = a1.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[0,start_ind:start_ind+20000],
           c=FTLEs[0,:20000],s=2,cmap='RdBu_r',vmax=10,vmin=-10)
a1.grid()
a1.set_xlabel('x')
a1.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('FTLE 1')

a2 = fig.add_subplot(5,1,2)
scat = a2.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[0,start_ind:start_ind+20000],
           c=FTLEs[1,:20000],s=2,cmap='RdBu_r',vmax=5,vmin=-5)
a2.grid()
a2.set_xlabel('x')
a2.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('FTLE 2')

a3 = fig.add_subplot(5,1,3)
scat = a3.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[0,start_ind:start_ind+20000],
           c=FTLEs[2,:20000],s=2,cmap='RdBu_r',vmax=1,vmin=-1)
a3.grid()
a3.set_xlabel('x')
a3.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('FTLE 3')

a3 = fig.add_subplot(5,1,4)
scat = a3.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[0,start_ind:start_ind+20000],
           c=FTLEs[3,:20000],s=2,cmap='RdBu_r',vmax=1,vmin=-1)
a3.grid()
a3.set_xlabel('x')
a3.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('FTLE 4')

a3 = fig.add_subplot(5,1,5)
scat = a3.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[0,start_ind:start_ind+20000],
           c=FTLEs[4,:20000],s=2,cmap='RdBu_r',vmax=1,vmin=-1)
a3.grid()
a3.set_xlabel('x')
a3.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('FTLE 5')

plt.show()

In [ ]:
np.mean(FTLEs,axis=1)

## CLVs

In [ ]:
M = 5
orth = 1
num_CLVs = cocycle.shape[2]-2*M

CLVs_filename = '.'.join(['CaputoCLVs', 'M{:d}'.format(M),'orth{:d}'.format(orth),'nc'])
CLVs_file = os.path.join(CLVS_DIR, CLVs_filename)

#if os.path.exists(CLVs_file):
#    CLVs = xr.open_dataset(CLVs_file).CLVs
#else:
    # Calculate CLVs
Nk = np.arange(0,M+1,orth) 

CLVs = np.zeros((dim,dim,num_CLVs),dtype=np.float64)

start = time.time()
for ti in np.arange(0,num_CLVs):
    CLVs[:,:,ti] = calculate_CLV_numerically(dim, cocycle[:,:,ti:ti+2*M], Nk, M)
end = time.time()

comp_time = (end-start)/60

print('calculated in ' + str(np.round(comp_time,4)) + ' min')
    
    ## convert to xarray
CLVs = xr.DataArray(CLVs, coords=[np.arange(1,CLVs.shape[0]+1),np.arange(1,CLVs.shape[1]+1), np.arange(M,M+num_CLVs)],
                               dims=['coordinate','CLV', 'timestep'])
    
    ## save to netCDF file
CLVs.to_dataset(name='CaputoCLVs').to_netcdf(CLVs_file)

In [ ]:
# Calculate alignment
align = np.array(np.zeros((dim,dim,num_CLVs),dtype=np.float64))

start = time.time()
for clvi in np.arange(0,dim):
    for clvj in np.arange(0,dim):
        for tt in np.arange(0,num_CLVs):
            align[clvi,clvj,tt] = abs(np.dot(CLVs.values[:,clvi,tt],CLVs.values[:,clvj,tt]))
            
end = time.time()
comp_time = (end-start)

print('alignment calculated in ' + str(np.round(comp_time,4)) + ' sec')

In [ ]:
fig = plt.figure(figsize=[24,30],layout='constrained',dpi=300)
dim2 = 5
for i in np.arange(0,dim2):
    for j in np.arange(0,i):
        a1 = fig.add_subplot(dim2,dim2,i*dim2+j+1,projection='3d')
        scat = a1.scatter(IC_run.x[0,start_ind+M:-M],IC_run.x[1,start_ind+M:-M],IC_run.x[2,start_ind+M:-M],
           c=align[i,j,:],s=2,cmap='RdBu_r',vmax=1,vmin=0) 
        a1.grid()
        a1.set_title('CLV{}{}'.format(i+1,j+1),fontsize=16)

cbar = plt.colorbar(scat,shrink=0.7) 
plt.savefig('Results/Caputo/alignment_xyz_M{}_orth{}.png'.format(M,orth),format="png",bbox_inches='tight')
plt.show()  

In [ ]:
fig = plt.figure(figsize=[24,30],layout='constrained',dpi=300)
dim2 = 5
for i in np.arange(0,dim2):
    for j in np.arange(0,i):
        a1 = fig.add_subplot(dim2,dim2,i*dim2+j+1,projection='3d')
        scat = a1.scatter(IC_run.x[1,start_ind+M:-M],IC_run.x[3,start_ind+M:-M],IC_run.x[4,start_ind+M:-M],
           c=align[i,j,:],s=2,cmap='RdBu_r',vmax=1,vmin=0) 
        a1.grid()
        a1.set_title('CLV{}{}'.format(i+1,j+1),fontsize=16)

cbar = plt.colorbar(scat,shrink=0.7) 
plt.savefig('Results/Caputo/alignment_ywu_M{}_orth{}.png'.format(M,orth),format="png",bbox_inches='tight')
plt.show()  

In [ ]:
fig = plt.figure(figsize=[24,30],layout='constrained',dpi=300)
dim2 = 5
for i in np.arange(0,dim2):
    for j in np.arange(0,i):
        a1 = fig.add_subplot(dim2,dim2,i*dim2+j+1,projection='3d')
        scat = a1.scatter(IC_run.x[0,start_ind+M:-M],IC_run.x[3,start_ind+M:-M],IC_run.x[4,start_ind+M:-M],
           c=align[i,j,:],s=2,cmap='RdBu_r',vmax=1,vmin=0) 
        a1.grid()
        a1.set_title('CLV{}{}'.format(i+1,j+1),fontsize=16)

cbar = plt.colorbar(scat,shrink=0.7) 
plt.savefig('Results/Caputo/alignment_xwu_M{}_orth{}.png'.format(M,orth),format="png",bbox_inches='tight')
plt.show()  

In [ ]:
fig = plt.figure(figsize=[24,30],layout='constrained',dpi=300)
a1 = fig.add_subplot(10,1,1)
scat = a1.scatter(IC_run.time[start_ind+8000:start_ind+12000],IC_run.x[0,start_ind+8000:start_ind+12000],
           c=align[0,1,8000:12000],s=3,cmap='RdBu_r',vmax=1,vmin=0)
a1.grid()
a1.set_xlabel('x')
a1.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('CLV 1&2')

a2 = fig.add_subplot(10,1,2)
scat = a2.scatter(IC_run.time[start_ind+8000:start_ind+12000],IC_run.x[0,start_ind+8000:start_ind+12000],
           c=align[0,2,8000:12000],s=3,cmap='RdBu_r',vmax=1,vmin=0)
a2.grid()
a2.set_xlabel('x')
a2.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('CLV 1&3')

a3 = fig.add_subplot(10,1,3)
scat = a3.scatter(IC_run.time[start_ind+8000:start_ind+12000],IC_run.x[0,start_ind+8000:start_ind+12000],
           c=align[0,3,8000:12000],s=3,cmap='RdBu_r',vmax=1,vmin=0)
a3.grid()
a3.set_xlabel('x')
a3.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('CLV 1&4')

a3 = fig.add_subplot(10,1,4)
scat = a3.scatter(IC_run.time[start_ind+8000:start_ind+12000],IC_run.x[0,start_ind+8000:start_ind+12000],
           c=align[0,4,8000:12000],s=3,cmap='RdBu_r',vmax=1,vmin=0)
a3.grid()
a3.set_xlabel('x')
a3.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('CLV 1&5')

a3 = fig.add_subplot(10,1,5)
scat = a3.scatter(IC_run.time[start_ind+8000:start_ind+12000],IC_run.x[0,start_ind+8000:start_ind+12000],
           c=align[1,2,8000:12000],s=3,cmap='RdBu_r',vmax=1,vmin=0)
a3.grid()
a3.set_xlabel('x')
a3.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('CLV 2&3')

a3 = fig.add_subplot(10,1,6)
scat = a3.scatter(IC_run.time[start_ind+8000:start_ind+12000],IC_run.x[0,start_ind+8000:start_ind+12000],
           c=align[1,3,8000:12000],s=3,cmap='RdBu_r',vmax=1,vmin=0)
a3.grid()
a3.set_xlabel('x')
a3.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('CLV 2&4')

a3 = fig.add_subplot(10,1,7)
scat = a3.scatter(IC_run.time[start_ind+8000:start_ind+12000],IC_run.x[0,start_ind+8000:start_ind+12000],
           c=align[1,4,8000:12000],s=3,cmap='RdBu_r',vmax=1,vmin=0)
a3.grid()
a3.set_xlabel('x')
a3.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('CLV 2&5')

a3 = fig.add_subplot(10,1,8)
scat = a3.scatter(IC_run.time[start_ind+8000:start_ind+12000],IC_run.x[0,start_ind+8000:start_ind+12000],
           c=align[2,3,8000:12000],s=3,cmap='RdBu_r',vmax=1,vmin=0)
a3.grid()
a3.set_xlabel('x')
a3.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('CLV 3&4')

a3 = fig.add_subplot(10,1,9)
scat = a3.scatter(IC_run.time[start_ind+8000:start_ind+12000],IC_run.x[0,start_ind+8000:start_ind+12000],
           c=align[2,4,8000:12000],s=3,cmap='RdBu_r',vmax=1,vmin=0)
a3.grid()
a3.set_xlabel('x')
a3.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('CLV 3&5')

a3 = fig.add_subplot(10,1,10)
scat = a3.scatter(IC_run.time[start_ind+8000:start_ind+12000],IC_run.x[0,start_ind+8000:start_ind+12000],
           c=align[3,4,8000:12000],s=3,cmap='RdBu_r',vmax=1,vmin=0)
a3.grid()
a3.set_xlabel('x')
a3.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('CLV 4&5')

plt.show()

In [ ]:
fig = plt.figure(figsize=[24,40],layout='constrained',dpi=300)
dim2 = 5
for i in np.arange(0,dim2):
    for j in np.arange(0,i):
        a1 = fig.add_subplot(dim2,dim2,i*dim2+j+1)
        scat = a1.scatter(IC_run.x[i,:75000],IC_run.x[j,:75000],
        c=align[i,j,:75000],s=2,cmap='RdBu_r',vmax=1,vmin=0)
        a1.grid()
        a1.set_title('{}-{} axes'.format(i+1,j+1),fontsize=24)

cbar = plt.colorbar(scat,shrink=0.7)
cbar.set_label('alignment',fontsize=24)
       
plt.savefig('Results/Caputo/alignment 2D axes view M{}_orth{}_wind{}.png'.format(M,orth,window),format="png",bbox_inches='tight')
plt.show()  

## Growth rates CLVs

In [ ]:
M_GR  = 5
orth_win = 1
Nk = np.arange(orth_win-1,M_GR,orth_win)
GRs = np.empty((CLVs.shape[1],CLVs.shape[2]))

start = time.time()

for i in np.arange(0,CLVs.shape[2]):
    C = np.array(CLVs[:,:,i])
    norm_C = linalg.norm(C,axis=0)
    GR_i = np.zeros((CLVs.shape[1],Nk.shape[0]))
    ni = 0
    for tt in np.arange(0,M_GR):
        C = np.matmul(cocycle[:,:,i+tt+M],C)
        if np.isin(tt,Nk):
            GR_i[:,ni] = (linalg.norm(C,axis=0)-norm_C) / (norm_C * dt * orth_win)
            norm_C = linalg.norm(C,axis=0)
            #C[:,1] = C[:,1] - (np.dot(C[:,0], C[:,1]) / np.dot(C[:,0], C[:,0]))*C[:,0]
            #C[:,2] = C[:,2] - (np.dot(C[:,1], C[:,2]) / np.dot(C[:,1], C[:,1]))*C[:,1]
            ni += 1
    #GRs[:,i] = (linalg.norm(C,axis=0)-norm_C)/norm_C
    #GRs[:,i] = GR_i[:,-1]
    GRs[:,i] = np.mean(GR_i,axis=1)

end = time.time()
elapsed = (end-start)/60
print("Elapsed time: {} min, M = {}".format(round(elapsed,4),M_GR)) 

In [ ]:
fig = plt.figure(figsize=[40,30],layout='constrained',dpi=300)

vlims = [[10,-10],[1,-1],[1,-1],[2,-2],[10,-10],[10,-10],[1,-1],[1,-1],[2,-2],[10,-10],[10,-10],[1,-1],[1,-1],[2,-2],[10,-10]]
dim2 = 5
for i in np.arange(0,dim2):
    a1 = fig.add_subplot(3,5,i+1,projection='3d')
    scat = a1.scatter(IC_run.x[0,start_ind+M:-M],IC_run.x[1,start_ind+M:-M],IC_run.x[2,start_ind+M:-M],
           c=GRs[i,:],s=2,cmap='RdBu_r',vmax=vlims[i][0],vmin=vlims[i][1])
    a1.grid()
    cbar = plt.colorbar(scat,shrink=0.7)
    a1.set_title('GR {}'.format(i+1),fontsize=32)
    
for i in np.arange(0,dim2):
    a1 = fig.add_subplot(3,5,i+6,projection='3d')
    scat = a1.scatter(IC_run.x[1,start_ind+M:-M],IC_run.x[2,start_ind+M:-M],IC_run.x[4,start_ind+M:-M],
           c=GRs[i,:],s=2,cmap='RdBu_r',vmax=vlims[i][0],vmin=vlims[i][1])
    a1.grid()
    cbar = plt.colorbar(scat,shrink=0.7)
    a1.set_title('GR {}'.format(i+1),fontsize=32)

for i in np.arange(0,dim2):
    a1 = fig.add_subplot(3,5,i+11,projection='3d')
    scat = a1.scatter(IC_run.x[0,start_ind+M:-M],IC_run.x[3,start_ind+M:-M],IC_run.x[4,start_ind+M:-M],
           c=GRs[i,:],s=2,cmap='RdBu_r',vmax=vlims[i][0],vmin=vlims[i][1])
    a1.grid()
    cbar = plt.colorbar(scat,shrink=0.7)
    a1.set_title('GR {}'.format(i+1),fontsize=32)

plt.savefig('Results/Caputo/FTCLEs_M{}_orth{}_M_GR{}_orient1.png'.format(M,orth,M_GR),format="png",bbox_inches='tight')
plt.show()

In [ ]:
np.mean(GRs,axis=1)

## Kaplan-Yorke Dimension

In [ ]:
#FTLE_type = 'FTBLEs'
FTLE_type = 'FTCLEs'

if FTLE_type == 'FTCLEs':
    KY_FTLEs = GRs
elif FTLE_type == 'FTBLEs':
    KY_FTLEs = FTLEs
else:
    print('error: KY_FTLEs type not defined')  
    
num_states = len(IC)
i_min = num_states-1
dimKY = np.empty((KY_FTLEs.shape[1]))
t = 0
    
start = time.time()
for tt in np.arange(0,KY_FTLEs.shape[1]):
    Lyaps_t = KY_FTLEs[:,tt]
    Lyaps_ord = np.sort(Lyaps_t)
    Lyaps_ord = Lyaps_ord[::-1]
    for ll in np.arange(1,Lyaps_ord.shape[0]):
        S = np.sum(Lyaps_ord[:ll])
        if S<0:
            i_min = ll
            break
        elif ll == dim:
            i_min = ll
    
    dimKY[t] = i_min + np.sum(Lyaps_ord[:i_min])/abs(Lyaps_ord[i_min])
    t += 1
    
end = time.time()
elapsed = end-start
print("Elapsed time: {} sec".format(round(elapsed,4),M))


In [ ]:
mycmap3 = matplotlib.colors.ListedColormap([
    [0.7, 0.6, 1., 1.      ],
    [0.7, 0.6, 1., 1.      ],
    [0.5, 0.3, 0.8, 1.     ],
    [0.5, 0.3, 0.8, 1.      ], 
    [0.3, 0., 0.6, 1.      ],
    [0.3, 0., 0.6, 1.]]) 

fig = plt.figure(figsize=[24,20],layout='constrained',dpi=300)
a1 = fig.add_subplot(1,3,1,projection='3d')

if FTLE_type == 'FTBLEs':
    scat = a1.scatter(IC_run.x[0,start_ind:-window],IC_run.x[1,start_ind:-window],IC_run.x[2,start_ind:-window],
           c=dimKY,s=2,cmap='RdBu_r',vmax=8,vmin=0)
elif FTLE_type == 'FTCLEs':
    scat = a1.scatter(IC_run.x[0,start_ind+M:-M],IC_run.x[1,start_ind+M:-M],IC_run.x[2,start_ind+M:-M],
           c=dimKY,s=2,cmap='RdBu_r',vmax=8,vmin=0)
else:
     print('error: KY_FTLEs type not defined')   
a1.grid()
cbar = plt.colorbar(scat,shrink=0.3)
cbar.set_label('dimKY')

a2 = fig.add_subplot(1,3,2,projection='3d')

if FTLE_type == 'FTBLEs':
    scat = a2.scatter(IC_run.x[1,start_ind:-window],IC_run.x[3,start_ind:-window],IC_run.x[4,start_ind:-window],
           c=dimKY,s=2,cmap='RdBu_r',vmax=8,vmin=0)
elif FTLE_type == 'FTCLEs':
    scat = a2.scatter(IC_run.x[1,start_ind+M:-M],IC_run.x[3,start_ind+M:-M],IC_run.x[4,start_ind+M:-M],
           c=dimKY,s=2,cmap='RdBu_r',vmax=8,vmin=0)
else:
     print('error: KY_FTLEs type not defined')   
a2.grid()
cbar = plt.colorbar(scat,shrink=0.3)
cbar.set_label('dimKY',fontsize=16)

a3 = fig.add_subplot(1,3,3,projection='3d')

if FTLE_type == 'FTBLEs':
    scat = a3.scatter(IC_run.x[0,start_ind:-window],IC_run.x[3,start_ind:-window],IC_run.x[4,start_ind:-window],
           c=dimKY,s=2,cmap='RdBu_r',vmax=8,vmin=0)
elif FTLE_type == 'FTCLEs':
    scat = a3.scatter(IC_run.x[0,start_ind+M:-M],IC_run.x[3,start_ind+M:-M],IC_run.x[4,start_ind+M:-M],
           c=dimKY,s=2,cmap='RdBu_r',vmax=8,vmin=0)
else:
     print('error: KY_FTLEs type not defined')   
a3.grid()
cbar = plt.colorbar(scat,shrink=0.3)
cbar.set_label('dimKY',fontsize=16)

if FTLE_type == 'FTBLEs':
    print('save command commented') 
    plt.savefig('Results/Caputo/dimKY_FTBLEs_wind{}.png'.format(window),format="png",bbox_inches='tight')
elif FTLE_type == 'FTCLEs':
    print('save command commented') 
    plt.savefig('Results/Caputo/dimKY_FTCLEs_M{}_M_GR_{}_wind{}_orth{}.png'.format(M,M_GR,window,orth_win),format="png",bbox_inches='tight')
else:
     print('error: KY_FTLEs type not defined')  
plt.show()

In [ ]:
fig = plt.figure(figsize=[24,18])
a1 = fig.add_subplot(5,1,1)
scat = a1.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[0,start_ind:start_ind+20000],
           c=dimKY[:20000],s=2,cmap='RdBu_r',vmax=8,vmin=0)
a1.grid()
a1.set_xlabel('time')
a1.set_ylabel('x')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('dimKY')

a2 = fig.add_subplot(5,1,2)
scat = a2.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[1,start_ind:start_ind+20000],
           c=dimKY[:20000],s=2,cmap='RdBu_r',vmax=8,vmin=0)
a2.grid()
a2.set_xlabel('time')
a2.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('dimKY')

a3 = fig.add_subplot(5,1,3)
scat = a3.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[2,start_ind:start_ind+20000],
           c=dimKY[:20000],s=2,cmap='RdBu_r',vmax=8,vmin=0)
a3.grid()
a3.set_xlabel('time')
a3.set_ylabel('z')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('dimKY')

a3 = fig.add_subplot(5,1,4)
scat = a3.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[3,start_ind:start_ind+20000],
           c=dimKY[:20000],s=2,cmap='RdBu_r',vmax=8,vmin=0)
a3.grid()
a3.set_xlabel('time')
a3.set_ylabel('w')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('dimKY')

a3 = fig.add_subplot(5,1,5)
scat = a3.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[4,start_ind:start_ind+20000],
           c=dimKY[:20000],s=2,cmap='RdBu_r',vmax=8,vmin=0)
a3.grid()
a3.set_xlabel('time')
a3.set_ylabel('u')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('dimKY')
plt.show()

In [ ]:
fig = plt.figure(figsize=[24,40],layout='constrained',dpi=300)
dim2 = 5
for i in np.arange(0,dim2):
    for j in np.arange(0,i):
        a1 = fig.add_subplot(dim2,dim2,i*dim2+j+1)
        scat = a1.scatter(IC_run.x[i,:75000],IC_run.x[j,:75000],
        c=dimKY[:75000],s=2,cmap='RdBu_r',vmax=10,vmin=0)
        a1.grid()
        a1.set_title('{}-{} axes'.format(i+1,j+1),fontsize=24)

cbar = plt.colorbar(scat,shrink=0.7)
cbar.set_label('dimKY',fontsize=24)
        
if FTLE_type == 'FTBLEs':
    print('save command commented') 
    plt.savefig('Results/Caputo/2D axes view dimKY_FTBLEs_wind{}.png'.format(window),format="png",bbox_inches='tight')
elif FTLE_type == 'FTCLEs':
    print('save command commented') 
    plt.savefig('Results/Caputo/2D axes view dimKY_FTCLEs_M{}_M_GR_{}_wind{}_orth{}.png'.format(M,M_GR,window,orth_win),format="png",bbox_inches='tight')
else:
     print('error: KY_FTLEs type not defined')  
plt.show()
 

In [ ]:
#25/0.